![Assimila logo](../helpers/logosmall.png "Assimila")
# Assimila Prise Products
# Analyse Rainfall Product

If the drop down menus do not appear restart the notebook by selecting "Kernel" then "Restart" above.

In [10]:
import os
import sys

sys.path.append('../')
from DQTools.DQTools import Dataset

sys.path.append('../helpers/')
from widgets import Widgets
from data import Data

import matplotlib

import matplotlib.cm as cm
import matplotlib.animation as animation

import matplotlib.cbook as cbook
import matplotlib.image as image

matplotlib.use('nbagg')
import matplotlib.pyplot as plt

import gdal
import numpy as np
import xarray as xr

import ipywidgets as widgets
import pandas as pd
import datetime
import calendar

from IPython.display import clear_output
from IPython.display import display

from IPython.core.display import display, HTML

# Change display width of the notebook to 100%
display(HTML("<style>.container { width:100% !important; }</style>"))
# Stops widget labels from being cut off by widget
display(HTML("<style>.widget-label { min-width: 20ex !important; }</style>"))

%matplotlib notebook 

def analyse_months(val):
    """
    Handle the min, mean and max for MONTHLY timestep.
    
    :returns: DataArrays for single month based on daily tmin, tmax
    """ 
#     _data = []
    
    num_days = calendar.monthrange(val.year, val.month)[1]
    days = [datetime.date(val.year, val.month, day) for day in range(1, num_days+1)]
    
    # Query DQTools for the selected month
    rfe_data = get_data(product='chirps', subproduct='rfe', start=days[0], stop=days[-1], 
                        region=[lat.value+1.5, lon.value+1.5, lat.value-1.5, lon.value-1.5])
    print(rfe_data)
#     for day in days:
#         _data.append(rfe_data.sel(time=day))
        
#     __data = xr.concat([d for d in __data], dim='time')  
    
    return rfe_data.mean('time')
      

def on_dropdown_dates_change(*args, add_colorbar=False):
    global rfe_data, data
    """
    Handles change event on dropdown_dates widget
    """
    if args:
        val = args[0]['new']
    else:
        val = dropdown_dates.value
    
    if val is None:
        return
    
    if timestep.value == "days":
        # TODO: query DQ and display color map for this selected date 
        rfe_data = get_data(product='chirps', subproduct='rfe', start=val, stop=val)
        data = rfe_data.sel(time=dropdown_dates.value)
    
    elif timestep.value == "months":
        data = analyse_months(val)
        print(data)
        data = rfe_data.sel(time=dropdown_dates.value)
    
#     vmin_vmax = [float(data.data), float(rfe.max().data)]
    
    colorbars = []
    rfe_p.clear()
    ts_p.clear()
    tmp_img = data.plot.imshow(ax=rfe_p, cmap='summer', add_colorbar=False)
                            #vmin=vmin_vmax[0], vmax=vmin_vmax[1])

    rfe_p.set_title('rfe')
    rfe_p.set_aspect('equal')
    if add_colorbar:
        colorbars.append(fig.colorbar(tmp_img, ax=rfe_p, orientation="horizontal", fraction=0.044, pad=0.1))
    rfe_p.grid()
    
    with out:
        fig.canvas.draw()
        plt.show()

        
def on_click(event):
    """
    Event handler
    """
    # Event does not apply for time series plot
    # Check if the click was in a
    ts_p.clear()

    # Delete last reference point
    if len(rfe_p.lines) > 0:
        del rfe_p.lines[0]

    # Draw a point as a reference
    rfe_p.plot(event.xdata, event.ydata,
            marker='o', color='red', markersize=7, alpha=0.7) 
    
    # Define a 'chip' of data to display to the user (9 square degrees)
    rfe_chip = rfe_data.where((rfe_data["latitude"] <lat.value+1.5)&
                              (rfe_data["latitude"] >lat.value-1.5)&
                              (rfe_data["longitude"]<lon.value+1.5)&
                              (rfe_data["longitude"]>lon.value-1.5), drop=True)
    
    # Plot the chip of data
    tmp_img = rfe_chip.plot(ax=rfe_chip_p, cmap='summer', add_colorbar=False)
    rfe_chip_p.grid()
    rfe_chip_p.set_visible(True)
    
def on_button_clicked(b):
    """
    Event handler function for 'Get location'
    """ 
    # Event does not apply for time series plot
    # Check if the click was in a
    ts_p.clear()
    
    # Delete last reference point
    if len(rfe_p.lines) > 0:
        del rfe_p.lines[0]
    
    # Draw a point as a reference
    rfe_p.plot(lon.value, lat.value,
            marker='o', color='red', markersize=7, alpha=0.7) 
    
    # Define a 'chip' of data to display to the user (9 square degrees)
    rfe_chip = data.where((rfe_data["latitude"] <lat.value+1.5)&
                              (rfe_data["latitude"] >lat.value-1.5)&
                              (rfe_data["longitude"]<lon.value+1.5)&
                              (rfe_data["longitude"]>lon.value-1.5), drop=True)
    
    # Plot the chip of data
    tmp_img = rfe_chip.plot(ax=rfe_chip_p, cmap='summer', add_colorbar=False)
    rfe_chip_p.grid()
    rfe_chip_p.set_visible(True)

def on_date_change(*args):
    """"
    Event handler function for changing selected date.
    """
    val = args[0]['new']   

    #dt_times = [pd.to_datetime(d).date() for d in times]
    on_timestep_change()
    
def on_timestep_change(*args):
    """
    Event handler function for changing timestep.
    """
    if args:
        val = args[0]['new']
    else:
        val = timestep.value
        
#     dt_times = [pd.to_datetime(d).date() for d in times]
#     dates = sorted(list(set([d for d in dt_times if start.value <= d <= end.value])))

    dates = [start.value+datetime.timedelta(days=x) for x in range((end.value-start.value).days)]
    if val == 'days':
        dropdown_dates.options = dates
        
    elif val == 'months':
        _months = pd.date_range(dates[0], dates[-1], freq='MS')
        dropdown_dates.options = [pd.to_datetime(d).date() for d in _months]
        
    elif val == 'years':
        dropdown_dates.options = pd.date_range(dates[0], dates[-1], freq='YS')

def on_data_button_clicked(b):
    """
    Event handler function for pressing 'Get data' button
    """
    button_data.disabled = True
    rfe_data = get_data(start=start.value, stop=end.value, region=[lat.value+1.5, lon.value+1.5, lat.value-1.5, lon.value-1.5])
    button_data.disabled = False
    
    # Event does not apply for time series plot
    # Check if the click was in a
    ts_p.clear()
    _ts = []

    if timestep.value == 'days':
        delta = end.value-start.value
        period = [start.value + datetime.timedelta(days=i) for i in range(delta.days)]
        for day in period: 
            _ts.append(rfe_data.sel(time=day))

    if timestep.value == 'months':
        period = pd.date_range(start.value, end.value, freq='MS')
        for i, month in enumerate(period):
            mean = analyse_months(month)
            _ts.append(mean)

    ts = xr.concat([d for d in _ts], dim='time')

    # Redefine x-coordinate values 
    ts.coords['time']  = ('time', period)
    
    # Draw a point as a reference
    rfe_p.plot(lon.value, lat.value,
            marker='o', color='red', markersize=7, alpha=0.7)

    _data = ts.sel(longitude=lon.value,
                    latitude=lat.value,
                    method='nearest')

    ts_p.bar(_data.time.data, _data.data, label=f'CHIRPS RFE',
                 color='blue', lw=1.5, alpha=0.5)

    ts_p.set_title('')
    tkw = dict(size=4, width=1.5)
    ts_p.tick_params(axis='y', **tkw)
    ts_p.set_ylabel('rfe (mm/day)')
    ts_p.grid()
    
    with out:
        fig.canvas.draw() 

In [11]:
############
# Get data #
############

def send_request(product, subproduct, start, stop, region):
    """
    Send a request for data to the DataCube using DQTools
    """
    ds = Dataset(product=product, 
                 subproduct=subproduct,
                 identfile=os.path.join(os.path.expanduser("~"),
                                        '.assimila_dq.txt'))
    
    ds.get_data(start=start, 
                stop=stop,
                region=region)
    
    
    return ds.data

# TODO: change to Africa tile for initialisation and region for chip call 
def get_data(product='chirps', subproduct='rfe', start=datetime.datetime(2009, 1, 1), 
             stop=datetime.datetime(2009, 1, 1), region=[40, 55, -40, -20]):
    """
    Call the DataCube request function above and get resultant data into correct
    format and extarct a list of times.
    """
    now = datetime.datetime.now()
    
    rfe_data = send_request(product, subproduct, start, stop, region)["rfe"]
    
    #print(f"Request runtime: {datetime.datetime.now()-now} seconds")
    #times = sorted(set([pd.to_datetime(d).date() for d in rfe_data.time.data]))
    
    return rfe_data

In [12]:
# Get initialisation data
rfe_data = get_data()

###########
# Widgets #
###########

w = Widgets()

freq = ['days', 'months']#, 'years']

lat = w.latitude()
lon = w.longitude()
lat.value, lon.value = -22, 32
button = widgets.Button(description='Get point',
                        disabled=False,
                        button_style='info',
                        tooltip='Click me to plot location data')

button_data = widgets.Button(description='Get data',
                        disabled=False,
                        button_style='info',
                        tooltip='Click me to retrieve selected data') 

start = w.get_date(value=datetime.date(2009,1,1), description='Start')
end = w.get_date(value=datetime.date(2009,12,31), description='End')

dropdown_dates = widgets.Dropdown(options=times,
                                  value=times[0],
                                  description='Date selection',
                                  disabled=False)

timestep = widgets.Dropdown(options=freq,
                            value=freq[0],
                            description='Timestep',
                            disabled=False)

# Define 2 layouts so that widgets can be hidden and shown 
# layout1 = widgets.Layout(width='auto', height='auto')
# layout2 = widgets.Layout(width='auto', height='auto')

# lat.layout = layout1
# lon.layout = layout1
# button.layout = layout1
# start.layout = layout2
# end.layout = layout2
# timestep.layout = layout2
# dropdown_dates.layout = layout2
# button_data.layout = layout2

# layout2.visibility = "hidden"

output_layout = widgets.Layout(width='100%', height='1600px')
out = w.display_output()
out.layout = output_layout

button.on_click(on_button_clicked)
button_data.on_click(on_data_button_clicked)

data_list = dropdown_dates.observe(on_dropdown_dates_change, names='value')
start.observe(on_date_change)
end.observe(on_date_change)
timestep.observe(on_timestep_change)

box1 = widgets.HBox([lat, lon, button])
box2 = widgets.HBox([start, end, timestep, dropdown_dates, button_data])
box = widgets.VBox([box1, box2])

with out:
    display(box)
    
    ################
    # Plot objects #
    ################
    
    fig = plt.figure(figsize=(16, 11))

    rfe_p = plt.subplot2grid((2, 3), (0, 1), colspan=1)
    
    rfe_chip_p = plt.subplot2grid((2, 3), (0, 2), colspan=1)
    
    rfe_chip_p.set_visible(False)
    
    plt.subplots_adjust(wspace=0.13, hspace=0.1)

    ts_p = plt.subplot2grid((2, 3), (1, 0), colspan=3)

# Connect the canvas with the event
cid = fig.canvas.mpl_connect('button_press_event', on_click)

images = []
on_timestep_change()
#on_dropdown_dates_change(add_colorbar=True)

NameError: name 'times' is not defined